In [ ]:
# Future improvements
# Create JSON array to assign IDs, 
#     keep track of PDF files process (each step?) etc.
# Implement 'fi'/'fl' check at beginning and end of words

In [1]:
# Imports
import os
import time, sys
import PyPDF2
from datetime import datetime, date
import decimal
import re
import spacy
from nltk.stem.snowball import SnowballStemmer
nlp = spacy.load("en_core_web_sm")
stemmer = SnowballStemmer(language='english')
txtFilesDir = 'Text Files'
absolute = 'C:/Users/micah/Documents/IWU/CIS Practicum/Files'

In [14]:
# Pre-condition: All PDF files to be processed are in the sub-directory
#     pdfDir, and pdfDir is in absPath. absPath is by default the 
#     directory in which the program is executed
# Post-condition: All PDF files processed without error are converted to
#     text files which are placed in a new sub-directory 'Text Files'
def pdfToText(pdfDir, absPath = os.getcwd(), txtDir = txtFilesDir):
    pdfDirectory = absPath+'/'+pdfDir
    txtDirectory = absPath+'/'+txtDir
    if pdfDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
# Creates 'Text Files' directory for converted PDFs
    if txtDir not in os.listdir(absPath):
        os.mkdir(txtDirectory)
    
    docNum = 0
    stopAt = 25
    totalNum = len([file for file in os.scandir(pdfDirectory) if file.name.endswith('.pdf')])
    with open(absPath+'/'+'log.txt', 'a+', encoding="utf-8") as log:
        log.write("PDF to Text\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")        
# Moves on to next entity if the current entity is not a PDF
        for entity in os.scandir(pdfDirectory):
            if not entity.name.endswith('.pdf') or entity.name[0] in '1234567890':
                continue
            docNum += 1
            index = -4 # Remove '.pdf' from file name when creating '.txt' file
            fileName = entity.name[:index]+'.txt'
            print("Now on '"+entity.name+"'. . . ", end='')
            
# This block attempts to read the PDF file, extract text from each page,
#     and write the text to a text file with the same name
# Some documents are protected, corrupted, etc. and text cannot be extracted
# Exceptions are recorded in log.txt
# hasError remains true until each step in the try block is complete
            if fileName not in os.listdir(txtDirectory): 
                hasError = True
                with open(pdfDirectory+'/'+entity.name, 'rb') as pdfFileObject:
                    try:
                        pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
                        numPages = pdfReader.getNumPages()
                        pageObject = pdfReader.getPage(0)
                        textFile = open(txtDirectory+'/'+fileName, 'a+', encoding="utf-8")
                        i = 0;
                        while i < numPages:
                            pageObject = pdfReader.getPage(i)
                            textFile.write(pageObject.extractText())
                            i += 1
                        print("done\n")
                        hasError = False
#                     except TypeError as e:
#                         log.write("TypeError: " + entity.name + ":\n" + str(e))
#                     except PyPDF2.utils.PdfReadError as e:
#                         log.write("PdfReadError: " + entity.name + ":\n" + str(e))
#                     except OSError as e:
#                         log.write("OSError: " + entity.name + ":\n" + str(e))
#                     except decimal.InvalidOperation as e:
#                         log.write("InvalidOperation: " + entity.name + ":\n" + str(e))
                    except Exception as e:
                        log.write(str(docNum)+": " + entity.name + ": \n\t" + str(e)+"\n")
                        textFile.close()
                        os.remove(txtDirectory+'/'+fileName)
                    pdfFileObject.close()

                if hasError:
                    print("there was an error reading this document. See log for details. Reference number "+str(docNum)+".\n")
            else:
                max = 10
                if len(fileName) > max:
                    print('"'+fileName[:11]+'"' + '...', end='')
                else:
                    print('"'+fileName+'"', end='')
                print(' already exists\n')
            if docNum == stopAt:
                print("PDF to Text was stopped after "+str(docNum)+" documents.")
                break
        log.write("\n\n")
pdfToText('PDF', absolute)

Now on 'A  cognitive affective model of organizational communication for designing IT.pdf'. . . "A  cognitiv"... already exists

Now on 'A Bible for the Disability Field.pdf'. . . "A Bible for"... already exists

Now on 'A Comparisoon of Binarization Methods for Historical Archive Documents.pdf'. . . "A Compariso"... already exists

Now on 'A light-weight text image processing method for handheld embedded cameras.pdf'. . . "A light-wei"... already exists

Now on 'A Mathematical Theory of Communication.pdf'. . . there was an error reading this document. See log for details. Reference number 5.

Now on 'A Model for Types and Levels of Human Interaction with Automation.pdf'. . . "A Model for"... already exists

Now on 'A Parser for Real-Time Speech Synthesis of Conversational Texts.pdf'. . . "A Parser fo"... already exists

Now on 'A Physically Based Approach to 2-D Shape Blending.pdf'. . . there was an error reading this document. See log for details. Reference number 8.

Now on 'A Revie

In [18]:
# Work on 10 (good) files at a time until pipeline works
#   then incrementally add files and clean up errors

# Function to remove \n
filesDir = 'Text Files'
def rmvN(filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    if filesDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    for entity in os.scandir(directory):
        with open(directory+'/'+entity.name, 'r+', encoding='utf-8') as f:
            text = f.read()
            text = re.sub('-\n', '', text)
            text = re.sub('\n', '', text)
            f.seek(0)
            f.write(text)
            f.truncate()
rmvN(filesDir, absolute)

In [19]:
# Funtion to move files without spaces to new 'Without Spaces' directory         
def checkSpaces(filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    if filesDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    wsDir = 'Without Spaces'
    if wsDir not in os.listdir(absPath):
        os.mkdir(absPath+'/'+wsDir)
        
    with open(absPath+'/'+'No Spaces.txt', 'a+', encoding='utf-8') as noSpaces: 
        noSpaces.write("Check Spaces\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")
        for entity in os.scandir(directory):
            f = open(directory+'/'+entity.name, 'r', encoding='utf-8')
            text = f.read()
            split = text.split(' ')
            if len(split) < len(text)/10 or text == '':
                f.close()
                noSpaces.write(entity.name+'\n')
                if entity.name not in os.listdir(absPath+'/'+wsDir):
                    os.rename(directory+'/'+entity.name, absPath+'/'+wsDir+'/'+entity.name)  
        noSpaces.write('\n\n')
checkSpaces(filesDir, absolute) 

In [20]:
# Function to remove stopwords
# NLTK or SpaCy
# Inverted File: gram:[doc1, doc3] or gram:[[doc1,freq], [doc3,freq]]
def rmvStopWords(nlp, filesDir = txtFilesDir, absPath = os.getcwd()):
    directory = absPath+'/'+filesDir
    exists = False
    for entity in os.scandir(absPath):
        if entity.is_dir() and entity.name == filesDir:
            exists = True
    if not exists:
        print('The specified directory "' + directory + '" does not exist')
        return
    for entity in os.scandir(absPath+'/'+filesDir):
        with open(directory+'/'+entity.name, 'r+', encoding='utf-8') as f:
            doc = nlp(f.read())
            noStopWords = [stemmer.stem(token.lemma_.lower()) for token in doc if not token.is_stop and not token.is_punct]
#             Quotation marks appear as 'fi' in text files (merged with words)
#             maybe compare to words that start with fi
            f.seek(0)
            f.write(" ".join(noStopWords))
            #                 f.write(token.orth_.lower() + ' ')
            f.truncate()

rmvStopWords(nlp, filesDir, absolute)

In [ ]:
# Function to stem words
    

In [ ]:
# Calculate cosine similarity score for each document

In [ ]:
# Trigram parser-based inverted file 
# (TF-DIF to remove trigrams common to most or all documents)

In [ ]:
# Clustering algorithm based on trigram inverted file

In [ ]:
# Add bigram parser-based info to inverted file

In [ ]:
# Implement clustering on bigram inverted file